In [130]:
import requests
import pandas as pd
import math
import aiohttp
import httpx
import asyncio
import ujson as json
from datetime import datetime
import time
import os
import sys
from typing import List, Optional, Dict, TypeAlias
JSON: TypeAlias = dict[str, "JSON"] | list["JSON"] | str | int | float | bool | None

from dotenv import load_dotenv
load_dotenv()

from CUSIP_Curve import CUSIP_Curve

import nest_asyncio
nest_asyncio.apply()

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
def cookie_string_to_dict(cookie_string):
    cookie_pairs = cookie_string.split('; ')
    cookie_dict = {pair.split('=')[0]: pair.split('=')[1] for pair in cookie_pairs if '=' in pair}
    return cookie_dict

In [60]:
t1 = time.time()
cookie_headers = {
    "authority": "services-dynarep.ddwa.finra.org",
    "method": "OPTIONS",
    "path": "/public/reporting/v2/data/group/FixedIncomeMarket/name/TreasuryTradeHistory",
    "scheme": "https",
    "accept": "*/*",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en-US,en;q=0.9",
    "access-control-request-headers": "content-type,x-xsrf-token",
    "access-control-request-method": "POST",
    "cache-control": "no-cache",
    "origin": "https://www.finra.org",
    "pragma": "no-cache",
    "priority": "u=1, i",
    "referer": "https://www.finra.org/",
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36"
}

cookie_url = "https://services-dynarep.ddwa.finra.org/public/reporting/v2/group/Firm/name/ActiveIndividual/dynamiclookup/examCode" 
cookie_response = requests.get(cookie_url, headers=cookie_headers)
cookie_str = dict(cookie_response.headers)["set-cookie"]
cookie_dict = cookie_string_to_dict(cookie_string=cookie_str)
print("cookie fetch time (sec): ", time.time() - t1)

t1 = time.time()
headers = {
    "authority": "services-dynarep.ddwa.finra.org",
    "method": "POST",
    "path": "/public/reporting/v2/data/group/FixedIncomeMarket/name/TreasuryTradeHistory",
    "scheme": "https",
    "accept": "application/json, text/plain, */*",
    "accept-encoding": "gzip, deflate, br, zstd",
    "accept-language": "en-US,en;q=0.9",
    "cache-control": "no-cache",
    # "content-length": str(sys.maxsize),
    "content-type": "application/json",
    "dnt": "1",
    "origin": "https://www.finra.org",
    "pragma": "no-cache",
    "priority": "u=1, i",
    "referer": "https://www.finra.org/",
    "sec-ch-ua": '"Not)A;Brand";v="99", "Google Chrome";v="127", "Chromium";v="127"',
    "sec-ch-ua-mobile": "?0",
    "sec-ch-ua-platform": '"Windows"',
    "sec-fetch-dest": "empty",
    "sec-fetch-mode": "cors",
    "sec-fetch-site": "same-site",
    "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/127.0.0.0 Safari/537.36",
    "x-xsrf-token": cookie_dict["XSRF-TOKEN"],
    "cookie": cookie_str 
}

payload = {
    "fields": [
        "issueSymbolIdentifier",
        "cusip",
        "tradeDate",
        "tradeTime",
        "reportedTradeVolume",
        "priceType",
        "lastSalePrice",
        "lastSaleYield",
        "reportingSideCode",
        "contraPartyTypeCode",
    ],
    "dateRangeFilters": [
        {
            "fieldName": "tradeDate", 
            "startDate": "2024-07-31", 
            "endDate": "2024-07-31"
        },
    ],
    "compareFilters": [
        {
            "fieldName": "cusip",
            "fieldValue": "912810UA4",
            "compareType": "EQUAL"
        },
        # {
        #     "fieldName": "tradeTime",
        #     "compareType": "GTE",
        #     "fieldValue": "08:00:00"
        # },
        # {
        #     "fieldName": "tradeTime",
        #     "compareType": "LTE",
        #     "fieldValue": "09:00:00"
        # }
    ],
    "limit": 5000,
    "offset": 50000,
    # "recordMaxLimit": sys.maxsize,
    # "responsePayloadMaxSize": "100mb",
    # "sortFields": ["-tradeTime"],
}

url = "https://services-dynarep.ddwa.finra.org/public/reporting/v2/data/group/FixedIncomeMarket/name/TreasuryTradeHistory"
res = requests.post(url, json=payload, headers=headers)
if res.ok:
    print(json.dumps(res.json()["returnBody"]["headers"], indent=4))
    trade_data_json = json.loads(res.json()["returnBody"]["data"])
    df = pd.DataFrame(trade_data_json)
    display(df)
    # df.to_excel("temp.xlsx")
else:
    print("res status: ", res.status_code)

print("data fetch time (sec): ", time.time() - t1)

cookie fetch time (sec):  0.5769882202148438
{
    "Record-Max-Limit": [
        "5000"
    ],
    "Transfer-Encoding": [
        "chunked"
    ],
    "Connection": [
        "keep-alive"
    ],
    "Pragma": [
        "no-cache"
    ],
    "Record-Limit": [
        "5000"
    ],
    "Date": [
        "Tue, 13 Aug 2024 21:44:55 GMT"
    ],
    "Record-Total": [
        "54649"
    ],
    "X-Frame-Options": [
        "DENY"
    ],
    "FINRA-api-request-id": [
        "0550f79f-08e9-4e97-8ec8-ad46bfc76258"
    ],
    "Response-Payload-Max-Size": [
        "3mb"
    ],
    "Record-Offset": [
        "50000"
    ],
    "Cache-Control": [
        "no-cache, no-store, max-age=0, must-revalidate"
    ],
    "Expires": [
        "0"
    ],
    "data-version": [
        "1"
    ],
    "Content-Type": [
        "application\/json"
    ]
}


,tradeTime,cusip,issueSymbolIdentifier,priceType,reportingSideCode,productSubTypeCode,contraPartyTypeCode,tradeDate,reportedTradeVolume,lastSaleYield,lastSalePrice
0,10:39:32,912810UA4,TSRYS5802849,D,S,NOTE,D,2024-07-31,1000000.00,4.348796,104.582031
1,10:39:32,912810UA4,TSRYS5802849,D,S,NOTE,D,2024-07-31,1000000.00,4.345381,104.640625
2,10:39:32,912810UA4,TSRYS5802849,D,B,NOTE,C,2024-07-31,1000000.00,4.345381,104.640625
3,10:39:32,912810UA4,TSRYS5802849,D,S,NOTE,D,2024-07-31,1000000.00,4.345381,104.640625
4,10:39:32,912810UA4,TSRYS5802849,D,B,NOTE,C,2024-07-31,1000000.00,4.345381,104.640625
...,...,...,...,...,...,...,...,...,...,...,...
4644,11:54:58,912810UA4,TSRYS5802849,D,S,NOTE,C,2024-07-31,1000000.00,4.347193,104.609525
4645,11:54:58,912810UA4,TSRYS5802849,D,S,NOTE,D,2024-07-31,3000000.00,4.347202,104.609375
4646,11:55:01,912810UA4,TSRYS5802849,D,S,NOTE,D,2024-07-31,1000000.00,4.347885,104.597656
4647,11:55:01,912810UA4,TSRYS5802849,D,S,NOTE,T,2024-07-31,1000000.00,4.347885,104.597656


data fetch time (sec):  11.938526630401611


In [126]:
def temp_tester_httpx(start, end, cusips):
    async def build_tasks(client: httpx.AsyncClient, start, end, cusips):
        tasks = await CUSIP_Curve._build_fetch_tasks_historical_trace_data(client=client, cusips=cusips, start_date=start, end_date=end)
        return await asyncio.gather(*tasks)
    
    async def run_fetch_all(start, end, cusips):
        async with httpx.AsyncClient() as client:
            all_data = await build_tasks(client=client, cusips=cusips, start=start, end=end)
            return all_data

    results = asyncio.run(run_fetch_all(cusips=cusips, start=start, end=end))
    return results

def temp_tester_aio(start, end, cusips):
    async def build_tasks(session: aiohttp.ClientSession, start, end, cusips):
        tasks = await CUSIP_Curve._build_fetch_tasks_historical_trace_data(session=session, cusips=cusips, start_date=start, end_date=end)
        return await asyncio.gather(*tasks)
    
    async def run_fetch_all(start, end, cusips):
        async with aiohttp.ClientSession() as session:
            all_data = await build_tasks(session=session, cusips=cusips, start=start, end=end)
            return all_data

    results = asyncio.run(run_fetch_all(cusips=cusips, start=start, end=end))
    return results

In [137]:
CUSIP_Curve(debug_verbose=True).fetch_historcal_trace_trade_history_by_cusip(
    cusips=["912810UA4"],
    start_date=datetime(2024, 7, 29),
    end_date=datetime(2024, 8, 2),
    # xlsx_path="trace_trade_history_may54s.xlsx",
)

DEBUG:urllib3.connectionpool:Starting new HTTPS connection (1): services-dynarep.ddwa.finra.org:443
DEBUG:urllib3.connectionpool:https://services-dynarep.ddwa.finra.org:443 "GET /public/reporting/v2/group/Firm/name/ActiveIndividual/dynamiclookup/examCode HTTP/1.1" 200 38
INFO:root:TRACE - FINRA Cookie Fetch Took: 0.556915283203125 seconds
INFO:root:TRACE - FINRA CUSIP API Payload Configs Took: 3.210221290588379 seconds
DEBUG:root:TRACE - CUSIP API Payload Configs: {'912810UA4': '233619'}
INFO:root:TRACE - Fetch All Took: 60.62942457199097 seconds
INFO:root:TRACE - DF Concation Took: 0.30578136444091797 seconds
INFO:root:TRACE - XLSX Write Took: 30.99504041671753 seconds
INFO:root:TRACE - Total Time Elapsed: 91.93332958221436 seconds


{'912810UA4':        tradeTime      cusip issueSymbolIdentifier priceType reportingSideCode  \
 0       08:00:06  912810UA4          TSRYS5802849         D                 S   
 1       08:00:06  912810UA4          TSRYS5802849         D                 S   
 2       08:00:06  912810UA4          TSRYS5802849         D                 S   
 3       08:00:06  912810UA4          TSRYS5802849         D                 S   
 4       08:00:08  912810UA4          TSRYS5802849         D                 S   
 ...          ...        ...                   ...       ...               ...   
 228614  17:30:01  912810UA4          TSRYS5802849         D                 B   
 228615  17:30:04  912810UA4          TSRYS5802849         D                 S   
 228616  17:31:41  912810UA4          TSRYS5802849         D                 S   
 228617  17:31:43  912810UA4          TSRYS5802849         D                 B   
 228618  17:35:56  912810UA4          TSRYS5802849         D                 S   
 
 